1. Install Dependencies

In [0]:
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 111.7 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


2. Configure Kaggle Credentials

In [0]:

import os

os.environ["KAGGLE_USERNAME"] = "devashishhsingh"
os.environ["KAGGLE_KEY"] = "KGAT_18d641616ff6fed25bcde54e5c6ea186"
print("Kaggle credentials configured!")

Kaggle credentials configured!


3. Create Database Schema

In [0]:

spark.sql("""
CREATE SCHEMA IF NOT EXISTS workspace.ecommerce
""")

DataFrame[]

4. Create Volume for Data Storage

In [0]:

spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.ecommerce_data
""")

DataFrame[]

5. Download Dataset from Kaggle

In [0]:

%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors


100%|██████████| 4.29G/4.29G [00:52<00:00, 87.1MB/s]


6. Extract Downloaded Dataset

In [0]:

%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
unzip -o ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

Archive:  ecommerce-behavior-data-from-multi-category-store.zip
  inflating: 2019-Nov.csv            
  inflating: 2019-Oct.csv            
total 18G
-rwxrwxrwx 1 spark-83cf1a4a-1e55-4d04-a92a-75 nogroup 8.4G Jan  9 15:47 2019-Nov.csv
-rwxrwxrwx 1 spark-83cf1a4a-1e55-4d04-a92a-75 nogroup 5.3G Jan  9 15:49 2019-Oct.csv
-rwxrwxrwx 1 spark-83cf1a4a-1e55-4d04-a92a-75 nogroup 4.3G Jan  9 15:45 ecommerce-behavior-data-from-multi-category-store.zip


7. Clean Up Zip File

In [0]:

%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
rm -f ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

total 14G
-rwxrwxrwx 1 spark-83cf1a4a-1e55-4d04-a92a-75 nogroup 8.4G Jan  9 15:47 2019-Nov.csv
-rwxrwxrwx 1 spark-83cf1a4a-1e55-4d04-a92a-75 nogroup 5.3G Jan  9 15:49 2019-Oct.csv


8. Restart Python Environment

In [0]:
%restart_python

9. Load November 2019 Data
10. Load October 2019 Data

In [0]:
df_n = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv",header=True,inferSchema=True)
df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv",header=True,inferSchema=True)


11. Display Dataset Statistics and Schema

In [0]:
print(f"October 2019 - Total Events: {df.count():,}")
print("\n" + "="*60)
print("SCHEMA:")
print("="*60)
df.printSchema()

October 2019 - Total Events: 42,448,764

SCHEMA:
root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



12. Display Sample Data

In [0]:
print("\n" + "="*60)
print("SAMPLE DATA (First 5 rows):")
print("="*60)
df.show(5, truncate=False)


SAMPLE DATA (First 5 rows):
+-------------------+----------+----------+-------------------+-----------------------------------+--------+-------+---------+------------------------------------+
|event_time         |event_type|product_id|category_id        |category_code                      |brand   |price  |user_id  |user_session                        |
+-------------------+----------+----------+-------------------+-----------------------------------+--------+-------+---------+------------------------------------+
|2019-10-01 00:00:00|view      |44600062  |2103807459595387724|NULL                               |shiseido|35.79  |541312140|72d76fde-8bb3-4e00-8c23-a032dfed738c|
|2019-10-01 00:00:00|view      |3900821   |2053013552326770905|appliances.environment.water_heater|aqua    |33.2   |554748717|9333dfbd-b87a-4708-9857-6336556b0fcc|
|2019-10-01 00:00:01|view      |17200506  |2053013559792632471|furniture.living_room.sofa         |NULL    |543.1  |519107250|566511c2-e2e3-422b-b695-c